In [1]:
# Define environment variables 
%env DEEP_ROOT=/content/drive/My Drive/Colab/Deep/brAIn-Power
%env DEEP_DATA=/content/drive/My Drive/Colab/Deep/DeepLearning/correct_data

# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory before importing files
import os
deep_root = os.getenv("DEEP_ROOT")
%cd $deep_root

!pip install import-ipynb
import import_ipynb

env: DEEP_ROOT=/content/drive/My Drive/Colab/Deep/brAIn-Power
env: DEEP_DATA=/content/drive/My Drive/Colab/Deep/DeepLearning/correct_data
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab/Deep/brAIn-Power


In [2]:
from LoadData import CustomDataset, load_data_2np
import UNet
import torch
import time
import numpy as np
from torch.utils.data import DataLoader,Dataset

importing Jupyter notebook from LoadData.ipynb
importing Jupyter notebook from UNet.ipynb


In [3]:
# Define some helper functions for later:
def IoU_score(target, prediction):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [4]:
# Load data the first time
#X_train, Y_train, X_valid, Y_valid, X_trans, Y_trans, X_test, Y_test = load_data_2np(hot_encoding=True, valid_perc=0.1, train_perc=0.1,show=0,CAD_perc=0,background=True)

In [5]:
# Save data for easier load next time - remember to name accodingly
# The naming scheme refers pparameters of the load_data_2np function
#np.save("data/X_train_hot_v0.1_t0.1_s0_c0_True", X_train)
#np.save("data/Y_train_hot_v0.1_t0.1_s0_c0_True", Y_train)
#np.save("data/X_valid_hot_v0.1_t0.1_s0_c0_True", X_valid)
#np.save("data/Y_valid_hot_v0.1_t0.1_s0_c0_True", Y_valid)
#np.save("data/X_trans_hot_v0.1_t0.1_s0_c0_True", X_trans)
#np.save("data/Y_trans_hot_v0.1_t0.1_s0_c0_True", Y_trans)
#np.save("data/X_test_hot_v0.1_t0.1_s0_c0_True", X_test)
#np.save("data/Y_test_hot_v0.1_t0.1_s0_c0_True", Y_test)

In [6]:
# Load dataset from .npy files
data_set = "hot_v0.1_t0.1_s0_c0_True"

X_train = np.load(f"data/X_train_{data_set}.npy")
Y_train = np.load(f"data/Y_train_{data_set}.npy")
X_valid = np.load(f"data/X_valid_{data_set}.npy")
Y_valid = np.load(f"data/Y_valid_{data_set}.npy")
X_test = np.load(f"data/X_test_{data_set}.npy")
Y_test = np.load(f"data/Y_trans_{data_set}.npy")

In [7]:
# Load transformations if needed
#X_trans = np.load(f"data/X_trans_{data_set}.npy")
#Y_trans = np.load(f"data/Y_trans_{data_set}.npy")

In [8]:
DS_train = CustomDataset(X_train, Y_train, one_hot=True)
DS_valid = CustomDataset(X_valid, Y_valid, one_hot=True)
DS_test = CustomDataset(X_test, Y_test, one_hot=True)

In [9]:
# Clear space
del X_train
del Y_train
del X_valid
del Y_valid
del X_test
del Y_test

In [10]:
# Perform preprocessing
#DS_train.gray_gamma_enhanced()
#DS_valid.gray_gamma_enhanced()
#DS_test.gray_gamma_enhanced()

In [11]:
# Check if GPU is available and functions to convert Tensors to Cuda
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")

Running GPU.


In [12]:
# Load in model
%%capture
net = UNet.UNet18(n_classes=9, rgb=True)
net.double()
if use_cuda:
    print('converting network to cuda-enabled')
    net.cuda()


In [13]:
# Converts tensors to cuda, if available
def get_variable(x):
    if use_cuda:
        return x.cuda()
    return x

In [14]:
# Get numpy array for both cuda and not
def get_numpy(x):
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [15]:
# Create lossfunction and optimizer
criterion = UNet.DiceLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=1e-3)

In [16]:
# Define hyper parameters
NUM_EPOCHS = 10
check_at = 1

In [17]:
# Define list to stor intermediat results
valid_iter = []
valid_loss = []
valid_iou = []

train_iter = []
train_loss = []
train_iou = []

# Define data loaders to serve data
DL_train = DataLoader(DS_train, batch_size=4, shuffle=True)
DL_valid = DataLoader(DS_valid)

#training loop
for epoch in range(NUM_EPOCHS):
    net.train()
    for i, data in enumerate(DL_train):
        input = data[0].type(torch.DoubleTensor)
        target = data[1].type(torch.DoubleTensor)

        # Tranfer to GPU if possible
        input = get_variable(input)
        target = get_variable(target)

        # Train the network
        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()

        # Store the results
        train_iter.append(epoch*len(DL_train)+i)
        train_loss.append(loss)

        # Save space
        del input
        del target
        del output
        del loss

    #validate
    if epoch%check_at==0:
        net.eval()
        L = 0
        for j,valid_data in enumerate(DL_valid):
            input = valid_data[0].type(torch.DoubleTensor)
            target = valid_data[1].type(torch.DoubleTensor)
            
            # Tranfer to GPU if possible
            input = get_variable(input)
            target = get_variable(target)
            
            output = net(input)
            loss = criterion(output, target)
            L += loss
            valid_iou.append(IoU_score(get_numpy(target),get_numpy(output)))

            # Save space
            del input
            del target
            del output
            del loss

        valid_iter.append(epoch)
        valid_loss.append(L/j)
        print(f'At epoch {epoch} Training loss is at {train_loss[-1]}')
        print(f'At epoch {epoch} Validation loss is at {valid_loss[-1]} and the IoU is at {valid_iou[-1]}%')

At epoch 0 Training loss is at 0.8179291886783429
At epoch 0 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 1 Training loss is at 0.8180734740470592
At epoch 1 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 2 Training loss is at 0.8179976334460277
At epoch 2 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 3 Training loss is at 0.818216766810314
At epoch 3 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 4 Training loss is at 0.818198465911121
At epoch 4 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 5 Training loss is at 0.8180026123661577
At epoch 5 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 6 Training loss is at 0.8181760659089465
At epoch 6 Validation loss is at inf and the IoU is at 0.1111111111111111%
At epoch 7 Training loss is at 0.8180806384513006
At epoch 7 Validation loss is at inf and the IoU is at 0.1111111111111111%
At